## Алгоритмы в биоинформатике
### ДЗ 2.1. Локальное выравнивание
#### Буклей Григорий

Задание: Реализуйте локальное выравнивание последовательностей. Придумайте две последовательности, для которых локальное и глобальное выравнивание с одинаковыми штрафами дают разные результаты (на участке локального выравнивания).

In [1]:
def local_alignment(seq1, seq2):
    d = [] # array fot dynamic
    prev = [] # array for fathers
    for i in range(len(seq1) + 1): # filling array with zeroes
        d.append([0] * (len(seq2) + 1))
        prev.append([(-1, -1)] * (len(seq2) + 1))

    for i in range(1, len(seq1) + 1):
        d[i][0] = 0
    for j in range(1, len(seq2) + 1):
        d[0][j] = 0

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if seq1[i - 1] == seq2[j - 1]:
                eq = 1
            else:
                eq = -1

            if d[i - 1][j - 1] + eq > 0:
                d[i][j] = d[i - 1][j - 1] + eq
                prev[i][j] = (i - 1, j - 1)

            if d[i - 1][j] > d[i][j]:
                d[i][j] = d[i - 1][j] - 1
                prev[i][j] = (i - 1, j)

            if d[i][j - 1] > d[i][j]:
                d[i][j] = d[i][j - 1] - 1
                prev[i][j] = (i, j - 1)

    ans1, ans2 = "", ""

    max_i_ind, max_j_ind = 0, 0
    max_elem = d[0][0]

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if d[i][j] > max_elem:
                max_elem = d[i][j]
                max_i_ind = i
                max_j_ind = j

    common_steps = min(len(seq1) - max_i_ind, len(seq2) - max_j_ind)


    for i in range(len(seq1) - 1 - common_steps, max_i_ind - 1, -1):
        ans1 += seq1[i]
        ans2 += "-"
    for j in range(len(seq2) - 1 - common_steps, max_j_ind - 1, -1):
        ans2 += seq2[j]
        ans1 += "-"
    for i in range(common_steps):
        ans1 += seq1[len(seq1) - i - 1]
        ans2 += seq2[len(seq2) - i - 1]

    curPosition = (max_i_ind, max_j_ind)
    while prev[curPosition[0]][curPosition[1]] != (-1, -1):
        if prev[curPosition[0]][curPosition[1]] == (curPosition[0] - 1, curPosition[1] - 1):
            ans1 += seq1[curPosition[0] - 1]
            ans2 += seq2[curPosition[1] - 1]
        elif prev[curPosition[0]][curPosition[1]] == (curPosition[0] - 1, curPosition[1]):
            ans1 += seq1[curPosition[0] - 1]
            ans2 += "-"
        else:
            ans1 += "-"
            ans2 += seq2[curPosition[1] - 1]

        curPosition = prev[curPosition[0]][curPosition[1]]

    common_steps = min(curPosition[0], curPosition[1])
    for i in range(common_steps):
        ans1 += seq1[curPosition[0] - i - 1]
        ans2 += seq2[curPosition[1] - i - 1]
    for j in range(curPosition[1] - 1 - common_steps, -1, -1):
        ans2 += seq2[j]
        ans1 += "-"
    for i in range(curPosition[0] - 1 - common_steps, -1, -1):
        ans1 += seq1[i]
        ans2 += "-"

    print(ans1[::-1])
    print(ans2[::-1])

Код нахождения глобального выравнивания для сравнения результатов выравниваний:

In [2]:
def global_alignment(seq1, seq2):
    d = [] # array fot dynamic
    prev = [] # array for fathers
    for i in range(len(seq1) + 1): # filling array with zeroes
        d.append([0] * (len(seq2) + 1))
        prev.append([(-1, -1)] * (len(seq2) + 1))

    for i in range(1, len(seq1) + 1):
        d[i][0] = -i
        prev[i][0] = (i - 1, 0)
    for j in range(1, len(seq2) + 1):
        d[0][j] = -j
        prev[0][j] = (0, j - 1)

    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if seq1[i - 1] == seq2[j - 1]:
                eq = 1
            else:
                eq = -1
            d[i][j] = d[i - 1][j - 1] + eq
            prev[i][j] = (i - 1, j - 1)

            if d[i - 1][j] > d[i][j]:
                d[i][j] = d[i - 1][j] - 1
                prev[i][j] = (i - 1, j)

            if d[i][j - 1] > d[i][j]:
                d[i][j] = d[i][j - 1] - 1
                prev[i][j] = (i, j - 1)

    ans1, ans2 = "", ""

    curPosition = (len(seq1), len(seq2))
    while curPosition != (0, 0):
        if prev[curPosition[0]][curPosition[1]] == (curPosition[0] - 1, curPosition[1] - 1):
            ans1 += seq1[curPosition[0] - 1]
            ans2 += seq2[curPosition[1] - 1]
        elif prev[curPosition[0]][curPosition[1]] == (curPosition[0] - 1, curPosition[1]):
            ans1 += seq1[curPosition[0] - 1]
            ans2 += "-"
        else:
            ans1 += "-"
            ans2 += seq2[curPosition[1] - 1]

        curPosition = prev[curPosition[0]][curPosition[1]]

    print(ans1[::-1])
    print(ans2[::-1])

Возьмём следующие строки


$$seq_1 = \text{KPOLKP}, \;\; seq_2 = \text{KQLKPO}$$

Тогда глобальным выравниванием для них является:
$$\text{K P O L K P _}$$ $$\text{K Q _ L K P O}$$

В то время как локальным будет:

$$\;\;\; \text{_ _ _ K P O L K P}$$ $$\text{K Q L K P O _ _ _}$$

In [3]:
print("Global:")
global_alignment("KPOLKP", "KQLKPO")
print()
print("Local:")
local_alignment("KPOLKP", "KQLKPO")

Global:
KPOLKP-
KQ-LKPO

Local:
---KPOLKP
KQLKPO---
